Crytals generated in params_200_50-20250314 resulted in some errors. Only 7954/10k of the crystals for n_arms=6 were created. This notebook's purpose is to diagnose the reason for these errors and to fix the bug in the crystal generation script.

In [22]:
# import packages
import numpy as np
import cadquery as cq
from scipy.stats import qmc
import math
import itertools
import matplotlib.pyplot as plt
import os
import json
import copy
import random
import sys
import miniball
import pandas as pd
import glob

# Source Code

In [2]:
### =========== (Rosette code) =========== ###
def create_bullet(a, c, hp, f_a, f_c, workplane):
    # create pyramid
    n_pyr = 6
    ri = a*np.cos(np.radians(30)) # distance between center and edge of hexagon
    theta = 90 - np.degrees(np.arctan(hp/ri))
    pyramid = workplane.polygon(n_pyr, f_a*2*a).extrude(-f_a*hp, taper=theta)
    # create cylinder 
    n_cyl = 6
    cylinder = workplane.polygon(n_cyl, f_a*2*a).extrude(f_c*2*c)
    # create bullet (union)
    bullet = cylinder.union(pyramid)
    return bullet

def calc_r0(f_r0, a, n_arms):
    '''
    linearly interpolate between perscribed limits for r0
    '''
    ymin, ymax = 0.5*a, 1*a
    xmin, xmax = 4, 12
    slope = (ymax-ymin)/(xmax-xmin)
    intercept = ymin - (slope*xmin)
    r0 = slope*(n_arms) + intercept
    r0 = f_r0 * r0 # multiply by perturbation factor
    return r0 

def calc_hp(f_hp, r0, n_arms):
    '''
    linearly interpolate: hp increases with n_arms
    '''
    ymin, ymax = 1*r0, 1.5*r0
    xmin, xmax = 4, 12
    slope = (ymax-ymin)/(xmax-xmin)
    intercept = ymin - (slope*xmin)
    hp = slope*(n_arms) + intercept
    hp = f_hp*hp # multiply by perturbation factot
    return hp

def calc_h0(f_h0, r0):
    '''
    h0 calculate as a perscribed fraction of r0
    '''
    h0 = r0/2
    h0 = f_h0*h0 # multiply by perturbation factor
    return h0

def extract_xyz(s_code):
    '''
    Convert list in format [x1, y1, z1, ..., xn, yn, zn] to separate x, y, z arrays
    '''
    x = []
    y = []
    z = []
    for i in range(0, len(s_code), 3):
        x.append(s_code[i])
        y.append(s_code[i+1])
        z.append(s_code[i+2])
    return x, y, z

# create_ros(base_params, n_arms, s_code, aspect_perturb)
def create_ros(params, n_arms, s_code, aspect_perturb):
    '''
    aspect_perturb: list in form [f_a_1,f_c_1,...,f_a_n_arms,f_c_n_arms]
    '''
    # unpack parameters
    a, c, f_r0, f_hp, f_h0 = params[0], params[1], params[2], params[3], params[4]
    r0 = calc_r0(f_r0, a, n_arms)
    hp = calc_hp(f_hp, r0, n_arms)
    h0 = calc_h0(f_h0, r0)
    # create sphere
    sphere = cq.Workplane().sphere(r0)
    # create outer shell to "place" bullets on
    # based on spherical code from Sloane et al. 
    r_outer = r0 + hp - h0
    # convert s_code list to outer_coords
    x, y, z = extract_xyz(s_code)
    outer_coords = r_outer*(np.column_stack((x, y, z)))
    # create and collect bullets in list
    bullets = []
    for i in range(len(outer_coords)):
        f_a = aspect_perturb[2*i]
        f_c = aspect_perturb[2*i + 1]
        normal_vector = tuple(outer_coords[i])
        plane = cq.Plane(origin=normal_vector, normal=normal_vector)
        workplane = cq.Workplane(plane)
        bullet = create_bullet(a, c, hp, f_a, f_c, workplane)
        bullets.append(bullet)
    # boolean union to create rosette
    ros = sphere.union(bullets[0])
    for i in range(1, n_arms):
        ros = ros.union(bullets[i])
    return ros
### ====================================== ###
def get_verts(ros, threshold):
    verts = ros.vertices() # list of vertices 
    origin = cq.Vertex.makeVertex(0,0,0)
    # filtered_verts = [v for v in verts if v.distance(origin) > threshold/2]
    filtered_verts = [v for v in verts]
    final_verts = np.asarray([list(v.Center().toTuple()) for v in filtered_verts])
    return final_verts 
    
    
def calc_mbs(points):
    """
    Calculate minimal bounding sphere (mbs)
    """
    mbs = {} # store attributes of sphere as dict

    # use miniball algorithm to find bounding sphere
    # mesh_points = np.asarray(points)
    unique_pts = np.unique(points, axis=0)
    c, r2 = miniball.get_bounding_ball(unique_pts)
    r = np.sqrt(r2) # r2 = radius squared, r = radius

    mbs['c'] = c # center coordinates as np array
    mbs['r'] = r # radius of sphere as float
    mbs['v'] = (4/3)*np.pi*(r**3)
    mbs['a'] = 4*np.pi*(r**2) 

    return mbs

def get_record(ros, params, id):
    try:
        sa = ros.val().Area()
        vol = ros.val().Volume()
        base_params = params[0]
        # print(f'rosette {id}: {base_params}')
        points = get_verts(ros, base_params[2])
        mbs = calc_mbs(points)
        rho_eff = vol/mbs['v'] 
        sa_eff = sa/mbs['a']
        record = [id]
        record.extend(base_params)
        record.extend([sa, vol, sa_eff, rho_eff])
        return record
    except Exception as e:
        print(f'rosette {id}: {base_params}')
        return f"An unexpected error occurred: {e}"

def process_instance(params, i, save_dir):
    # extract params
    base_params = params[0][:5]
    n_arms = params[0][5]
    aspect_perturb = params[1]
    s_code = params[2]
    ros = create_ros(base_params, n_arms, s_code, aspect_perturb)
    # make stl and record dirs if they don't exist
    record_dir = save_dir + f'/data/{n_arms}'
    stl_dir = save_dir + f'/stl/{n_arms}'
    os.makedirs(record_dir, exist_ok=True)
    os.makedirs(stl_dir, exist_ok=True)
    # calc attributes and save record as txt
    record = get_record(ros, params, i)
    record_filename = f'record-ros-test-{i:06d}.txt'
    record_filepath = os.path.join(record_dir, record_filename)
    print(record_filepath)
    with open(record_filepath, 'w') as file:
        file.write(",".join(map(str, record))) 
    # save model
    save_filename = f'ros-test-{i:06d}.stl'
    save_filepath = os.path.join(stl_dir, save_filename)
    print(save_filepath)
    print(type(ros))
    cq.exporters.export(ros, save_filepath) # save file

def process_chunk(chunk, start_index, end_index, save_dir):
    for i in range(start_index, end_index):
        p = chunk[i]
        process_instance(p, i, save_dir)

# Main

## Identify error

Generate n_arms=6 crystals, and identify parameter sets that result in error.

In [19]:
# read test subset of params
params_path = '/glade/u/home/joko/ice3d/output/params_200_50.json'
# load saved json
with open(params_path, 'rb') as file:
    params = json.load(file)

In [16]:
for i in range(100):
    params = param_list[72]
    base_params = params[0][:5]
    n_arms = params[0][5]
    aspect_perturb = params[1]
    s_code = params[2]
    ros = create_ros(base_params, n_arms, s_code, aspect_perturb)
    points = get_verts(ros, base_params[2])
    mbs = calc_mbs(points)

In [14]:
points = get_verts(ros, base_params[2])
mbs = calc_mbs(points)
mbs

{'c': array([-1.01599954,  4.37941962, -2.62988389]),
 'r': np.float64(104.07937741335735),
 'v': np.float64(4722616.350027278),
 'a': np.float64(136125.42082965572)}

In [15]:
record_list[72]

'An unexpected error occurred: Singular matrix'

In [52]:
data_dir_n10 = '/glade/derecho/scratch/joko/synth-ros/params_200_50-20250314/data/4'
# Use glob to find all .txt files in the directory
file_paths = glob.glob(f"{data_dir_n10}/*.txt")
# Read all the files into a list of DataFrames and concatenate them
# df = pd.concat([pd.read_csv(file, header=None) for file in file_paths], ignore_index=True)

# Initialize an empty list to hold the arrays
data_list = []

count = 0
# Read each file with numpy.loadtxt and append to the list
for file in file_paths:
    with open(file, 'r') as f:
        data = f.read()  # Read all lines into a list
        if count %1000==0:
            print(file)
            print(data)
        # data = data.split(',')
        # data = np.array([float(num.strip()) for num in data])
        data_list.append(data)
    count+=1

# # Step 1: Split the string by commas and remove any extra spaces
# num_list = num_str.split(',')

# # Step 2: Convert the list of strings to floats (or ints if you prefer integers)
# num_array = np.array([float(num.strip()) for num in num_list])

# Stack all numpy arrays vertically
# data_combined = np.vstack(data_list)

# df = pd.DataFrame(data_combined)
# df.head()

# colnames = ['id', 'a', 'c', 'f_r0', 'f_hp', 'f_h0', 'n_arms', 'sa', 'vol', 'sa_eff', 'rho_eff']
# df.columns = colnames
# df.head()

/glade/derecho/scratch/joko/synth-ros/params_200_50-20250314/data/4/record-ros-test-009499.txt
9499,22.877679439096593,73.63939254263525,1.2010828689522794,0.8418152008850748,0.9104039609734568,4,85457.44389445365,738205.467669833,0.2032254974827413,0.028790280056762752
/glade/derecho/scratch/joko/synth-ros/params_200_50-20250314/data/4/record-ros-test-001429.txt
1429,26.65054782205449,112.30636300747247,1.02874891175127,0.8830768710917517,0.8754943598386951,4,153526.98729904427,1639596.8763778184,0.20847871156484707,0.027591718397094022
/glade/derecho/scratch/joko/synth-ros/params_200_50-20250314/data/4/record-ros-test-005625.txt
5625,32.40600152235378,173.7645464116008,0.970878012047676,0.9781211082840617,0.8293139237663365,4,277345.75839880836,3665392.343238653,0.16412930189009098,0.017745716778146643
/glade/derecho/scratch/joko/synth-ros/params_200_50-20250314/data/4/record-ros-test-002926.txt
2926,29.002788082615055,67.2892975222837,0.9553009332623589,1.0789774104851735,0.95938947

In [54]:
count = 0
for data in data_list:
    if 'A' in data:
        # print(data)
        count+=1
print(count)

14


In [20]:
record_list = []
param_list = []
id = 0
for p in params:
     # extract params
    base_params = p[0][:5]
    n_arms = p[0][5]
    aspect_perturb = p[1]
    s_code = p[2]
    if n_arms==6:
        ros = create_ros(base_params, n_arms, s_code, aspect_perturb)
        param_list.append(p)
        record = get_record(ros, p, id)
        record_list.append(record)
        print(id)
        if id == 999:
            break
        id+=1 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27